# Imports

In [ ]:
import msk_modelling_python as msk
import matplotlib.pyplot as plt
import numpy as np
import os
import opensim as osim
from xml.etree import ElementTree as ET
current_dir = os.getcwd()
print(current_dir)

# Rename files

In [ ]:
folder = r"C:\Git\opensim_tutorial\tutorials\repeated_sprinting\Simulations\PC006\trial2_r1"
file_mapping = {
    'Patient_06_': '',
    
}
# loop through all files in the folder and subfolders
for root, dirs, files in os.walk(folder):
    for file in files:
        if any(substring in file for substring in file_mapping.keys()):
            print(file)
            new_file = file.replace(list(file_mapping.keys())[0], list(file_mapping.values())[0])
            print(os.path.join(root, file))
            
            os.rename(os.path.join(root, file), os.path.join(root, new_file))
            print(f"Renamed {file} to {new_file}")

Patient_06_MuscleAnalysis_ActiveFiberForce.sto
C:\Git\opensim_tutorial\tutorials\repeated_sprinting\Simulations\PC006\trial2_r1\Results_SO_and_MA\Patient_06_MuscleAnalysis_ActiveFiberForce.sto
Renamed Patient_06_MuscleAnalysis_ActiveFiberForce.sto to MuscleAnalysis_ActiveFiberForce.sto
Patient_06_MuscleAnalysis_ActiveFiberForceAlongTendon.sto
C:\Git\opensim_tutorial\tutorials\repeated_sprinting\Simulations\PC006\trial2_r1\Results_SO_and_MA\Patient_06_MuscleAnalysis_ActiveFiberForceAlongTendon.sto
Renamed Patient_06_MuscleAnalysis_ActiveFiberForceAlongTendon.sto to MuscleAnalysis_ActiveFiberForceAlongTendon.sto
Patient_06_MuscleAnalysis_FiberActivePower.sto
C:\Git\opensim_tutorial\tutorials\repeated_sprinting\Simulations\PC006\trial2_r1\Results_SO_and_MA\Patient_06_MuscleAnalysis_FiberActivePower.sto
Renamed Patient_06_MuscleAnalysis_FiberActivePower.sto to MuscleAnalysis_FiberActivePower.sto
Patient_06_MuscleAnalysis_FiberForce.sto
C:\Git\opensim_tutorial\tutorials\repeated_sprinting\S

# Create classes and functions

In [39]:

class File:
    def __init__(self, path):
        self.path = path
        self.name = os.path.basename(path)
        self.extension = os.path.splitext(path)[1]
        
        if not os.path.isfile(path):
            print(f"\033[93mFile not found: {path}\033[0m")
            return        
        
        try:
            endheader_line = msk.classes.osimSetup.find_file_endheader_line(path)
        except:
            print(f"Error finding endheader line for file: {path}")
            endheader_line = 0
        # Read file based on extension
        try:
            if self.extension == '.csv':
                self.data = msk.pd.read_csv(path)
            elif self.extension == '.json':
                self.data = msk.bops.import_json_file(path)
            elif self.extension == '.xml':
                self.data = msk.bops.XMLTools.load(path)
            else:
                try:
                    self.data = msk.pd.read_csv(path, sep="\t", skiprows=endheader_line)
                except:
                    self.data = None
                    
            # add time range for the data
            try:
                self.time_range = [self.data['time'].iloc[0], self.data['time'].iloc[-1]]
                try:
                    self.time_range = [self.data['Time'].iloc[0], self.data['Time'].iloc[-1]]
                except:
                    pass
            except:
                self.time_range = None
        
        except Exception as e:
            print(f"Error reading file: {path}")
            print(e)
            self.data = None
            self.time_range = None
            
class Trial:
    '''
    Class to store trial information and file paths, and export files to OpenSim format
    
    Inputs: trial_path (str) - path to the trial folder
    
    Attributes:
    path (str) - path to the trial folder
    name (str) - name of the trial folder
    og_c3d (str) - path to the original c3d file
    c3d (str) - path to the c3d file in the trial folder
    markers (str) - path to the marker trc file
    grf (str) - path to the ground reaction force mot file
    ...
    
    Methods: use dir(Trial) to see all methods
    
    '''
    def __init__(self, trial_path):        
        self.path = trial_path
        self.name = os.path.basename(self.path)
        self.subject = os.path.basename(os.path.dirname(self.path))
        self.c3d = os.path.join(os.path.dirname(self.path), self.name + '.c3d')
        self.markers = File(os.path.join(self.path,'markers_experimental.trc'))
        self.grf = File(os.path.join(self.path,'Visual3d_SIMM_grf.mot'))
        self.emg = File(os.path.join(self.path,'processed_emg.mot'))
        self.ik = File(os.path.join(self.path,'Visual3d_SIMM_input.mot'))
        self.id = File(os.path.join(self.path,'inverse_dynamics.sto'))
        self.so_force = File(os.path.join(self.path,'Results_SO_and_MA', f'{self.subject}_StaticOptimization_force.sto'))
        self.so_activation = File(os.path.join(self.path, 'Results_SO_and_MA', f'{self.subject}_StaticOptimization_activation.sto'))
        self.jra = File(os.path.join(self.path,'joint_reacton_loads.sto'))
        
        # load muscle analysis files
        self.ma_targets = ['_MomentArm_', '_Length.sto']
        self.ma_files = []
        try:
            files = os.listdir(os.path.join(self.path, 'Results_SO_and_MA'))
            for file in files:
                if file.__contains__(self.ma_targets[0]) or file.__contains__(self.ma_targets[1]):
                    self.ma_files.append(File(os.path.join(self.path, 'Results_SO_and_MA', file)))
        except:
            self.ma_files = None
                    
        # settings files
        self.grf_xml = File(os.path.join(self.path,'GRF_Setup.xml'))
        self.actuators_so = File(os.path.join(self.path,'actuators_SO.xml'))
        
        self.settings_json = File(os.path.join(self.path,'settings.json'))
                             
    def check_files(self):
        '''
        Output: True if all files exist, False if any file is missing
        '''
        files = self.__dict__.values()
        all_files_exist = True
        for file in files:
            try:
                if not os.path.isfile(file):
                    print('File not found: ' + file)
                    all_files_exist = False
            except:
                pass
        return all_files_exist
    
    def create_settings_json(self, overwrite=False):
        if os.path.isfile(self.settings_json) and not overwrite:
            print('settings.json already exists')
            return
        
        settings_dict = self.__dict__
        msk.bops.save_json_file(settings_dict, self.settings_json)
        print('trial settings.json created in ' + self.path)
    
    def exportC3D(self):
        msk.bops.c3d_osim_export(self.og_c3d) 

    def create_grf_xml(self):
        msk.classes.osimSetup.create_grf_xml(self.grf, self.grf_xml)

    def save_json_file(self, data, jsonFilePath):
        data = data.__dict__

        with open(jsonFilePath, 'w') as f:
            json.dump(data, f, indent=4)

        json_data = import_json_file(jsonFilePath)
        return json_data
    
    def to_json(self):
        msk.bops.save_json_file(self.__dict__, jsonFilePath = self.settings_json)
        print('settings.json created in ' + self.settings_json)
    
    def run_IK(osim_modelPath, trc_file, resultsDir):
        '''
        Function to run Inverse Kinematics using the OpenSim API.
        
        Inputs:
                osim_modelPath(str): path to the OpenSim model file
                trc_file(str): path to the TRC file
                resultsDir(str): path to the directory where the results will be saved
        '''

        # Load the TRC file
        import pdb; pdb.set_trace()
        tuple_data = import_trc_file(trc_file)
        df = pd.DataFrame.from_records(tuple_data, columns=[x[0] for x in tuple_data])
        column_names = [x[0] for x in tuple_data]
        if len(set(column_names)) != len(column_names):
            print("Error: Duplicate column names found.")
        # Load the model
        osimModel = osim.Model(osim_modelPath)                              
        state = osimModel.initSystem()

        # Define the time range for the analysis
        
        initialTime = TRCData.getIndependentColumn()
        finalTime = TRCData.getLastTime()

        # Create the inverse kinematics tool
        ikTool = osim.InverseKinematicsTool()
        ikTool.setModel(osimModel)
        ikTool.setStartTime(initialTime)
        ikTool.setEndTime(finalTime)
        ikTool.setMarkerDataFileName(trc_file)
        ikTool.setResultsDir(resultsDir)
        ikTool.set_accuracy(1e-6)
        ikTool.setOutputMotionFileName(os.path.join(resultsDir, "ik.mot"))

        # print setup
        ikTool.printToXML(os.path.join(resultsDir, "ik_setup.xml"))         

        # Run inverse kinematics
        print("running ik...")                                             
        ikTool.run()

    def run_inverse_kinematics(model_file, marker_file, output_motion_file):
        # Load model and create an InverseKinematicsTool
        model = osim.Model(model_file)
        ik_tool = osim.InverseKinematicsTool()

        # Set the model for the InverseKinematicsTool
        ik_tool.setModel(model)

        # Set the marker data file for the InverseKinematicsTool
        ik_tool.setMarkerDataFileName(marker_file)

        # Specify output motion file
        ik_tool.setOutputMotionFileName(output_motion_file)

        # Save setup file
        ik_tool.printToXML('setup_ik.xml')

        # Run Inverse Kinematics
        ik_tool.run()

    def run_ID(self, osim_modelPath, coordinates_file, external_loads_file, output_file, LowpassCutoffFrequency=6, run_tool=True):
        
        try: 
            model = osim.Model(osim_modelPath)
        except Exception as e:
            print(f"Error loading model: {osim_modelPath}")
            print(e)
            return
        
        results_folder = os.path.dirname(output_file)
        
        # Setup for excluding muscles from ID
        exclude = osim.ArrayStr()
        exclude.append("Muscles")
        # Setup for setting time range
        IKData = osim.Storage(coordinates_file)

        # Create inverse dynamics tool, set parameters and run
        id_tool = osim.InverseDynamicsTool()
        id_tool.setModel(model)
        id_tool.setCoordinatesFileName(coordinates_file)
        id_tool.setExternalLoadsFileName(external_loads_file)
        id_tool.setOutputGenForceFileName(output_file)
        id_tool.setLowpassCutoffFrequency(LowpassCutoffFrequency)
        id_tool.setStartTime(IKData.getFirstTime())
        id_tool.setEndTime(IKData.getLastTime())
        id_tool.setExcludedForces(exclude)
        id_tool.setResultsDir(results_folder)
        id_tool.printToXML(os.path.join(results_folder, "setup_ID.xml"))
        
        if run_tool:
            id_tool.run()



class Subject:
    def __init__(self, subject_path=None, model_path=None):
        
        if not subject_path: 
            print('Subject path not provided'); return
        
        self.path = subject_path
        self.name = os.path.basename(self.path)
        self.trials = []
        if not model_path: 
            print('Model path not provided')
            self.model = os.path.join(self.path, self.name + '_scaled.osim')
            print(f'Using default model path: {self.model}')
        else:
            self.model = model_path
        
    def load_trials(self, trial_names = ['trial1','trial2','trial3'], trial_number = 1):
        self.trials = []
        for trial in trial_names:
            trial_path = os.path.join(self.path, f'{trial}_{trial_number}')
            self.trials.append(Trial(trial_path))
            
    def to_json(self):
        msk.bops.save_json_file(self.__dict__, jsonFilePath = os.path.join(self.path, 'settings.json'))
        print('settings.json created in ' + self.path)
            
class openSim:
    def __init__(self, leg = 'r', subjects =['PC002','PC006','PC013'], trials_to_load = ['trial1','trial2','trial3'], trial_number = 1):
        try:
            self.code_path = os.path.dirname(__file__)
        except:
            self.code_path = os.getcwd()
        
        self.simulations_path = os.path.join(os.path.dirname(self.code_path), 'Simulations')
        self.subjects = {}
        
        for subject in subjects:
            self.subjects[subject] = {}
            self.subjects[subject]['model'] = os.path.join(self.simulations_path, subject, subject + '_scaled.osim')
            
            for trial in trials_to_load:                
                self.trial_path = os.path.join(self.simulations_path, subject, f'{trial}_{leg}{trial_number}')
                try:
                    trial = Trial(self.trial_path)
                    self.subjects[subject][trial.name] = trial 
                except Exception as e:
                    self.subjects[subject][trial] =  None
                    # print(f"Error loading trial: {self.trial_path}")
                    # print(e)
        

        self.ik_columns = ["hip_flexion_" + leg, "hip_adduction_" + leg, "hip_rotation_" + leg, "knee_angle_" + leg, "ankle_angle_" + leg]
        self.id_columns = ["hip_flexion_" + leg + "_moment", "hip_adduction_" + leg + "_moment", "hip_rotation_" + leg + "_moment", "knee_angle_" + leg + "_moment", "ankle_angle_" + leg + "_moment"]
        self.force_columns = ["add_long_" + leg, "rect_fem_" + leg, "med_gas_" + leg, "semiten_" + leg,"tib_ant_" + leg]


        self.titles = ["Hip Flexion", "Hip Adduction", "Hip Rotation", "Knee Flexion", "Ankle Plantarflexion"]
        self.titles_muscles = ["Adductor Longus", "Rectus Femoris", "Medial Gastrocnemius", "Semitendinosus", "Tibialis Anterior"]

    # Time Normalisation Function 
    def time_normalised_df(self, df, fs=None):
        if not isinstance(df, msk.pd.DataFrame):
            raise Exception('Input must be a pandas DataFrame')
        
        if not fs:
            try:
                fs = 1 / (df['time'][1] - df['time'][0])  # Ensure correct time column
            except KeyError:
                raise Exception('Input DataFrame must contain a column named "time"')
            
        normalised_df = msk.pd.DataFrame(columns=df.columns)

        for column in df.columns:
            if column == 'time':  # Skip time column
                continue	
            normalised_df[column] = msk.np.zeros(101)

            currentData = df[column].dropna()  # Remove NaN values

            timeTrial = msk.np.linspace(0, len(currentData) / fs, len(currentData))  # Original time points
            Tnorm = msk.np.linspace(0, timeTrial[-1], 101)  # Normalize to 101 points

            normalised_df[column] = msk.np.interp(Tnorm, timeTrial, currentData)  # Interpolate

        return normalised_df

    def plot_single_trial(self, show = False):
        #Read .mot files
        with open(self.mot_file, "r") as file:
            lines = file.readlines()

        # Find the line where actual data starts (usually after 'endheader')
        for i, line in enumerate(lines):
            if "endheader" in line:
                start_row = i + 1  # Data starts after this line
                break
        else:
            start_row = 0  # If 'endheader' is not found, assume no header

        # Load data using Pandas
        self.df_ik = msk.pd.read_csv(self.mot_file, delim_whitespace=True, start_row=start_row)
        self.df_id = msk.pd.read_csv(self.id_file, sep="\t", start_row=6)
        self.df_force = msk.pd.read_csv(self.force_file, sep="\t", start_row=14)

        # Apply normalisation to both IK (angles) and ID (moments) data
        self.df_ik_normalized = self.time_normalised_df(df=self.df_ik)
        self.df_id_normalized = self.time_normalised_df(df=self.df_id)
        self.df_force_normalized = self.time_normalised_df(df=self.df_force)

        # Ensure time is normalized to 101 points
        time_normalized = msk.np.linspace(0, 100, 101)  
 
        # select the specified columns         
        self.ik_data = self.df_ik_normalized[self.ik_columns]
        self.id_data = self.df_id_normalized[self.id_columns]
        self.force_data = self.df_force_normalized[self.force_columns]
            
        # Define the layout 
        fig, axes = plt.subplots(2, 5, figsize=(15, 4)) 

        #Plot IK (angles)
        for i, col in enumerate(self.ik_columns):
            ax = axes[0,i]
            ax.plot(time_normalized, self.ik_data[col], color='red')  # Main curve
            ax.set_title(self.titles[i])
            if i == 0:
                ax.set_ylabel("Angle (deg)")
            ax.grid(True)

        #Plot ID (moments)
        for i, col in enumerate(self.id_columns):
            ax = axes[1,i]
            ax.plot(time_normalized, self.id_data[col], color='blue')  # Main curve
            ax.set_title(self.titles[i])
            if i == 0:
                ax.set_ylabel("Moment (Nm)")
            ax.set_xlabel("% Gait Cycle")
            ax.grid(True)

        plt.tight_layout()


        # PLOT MUSCLE FORCES 
        fig, axes = plt.subplots(nrows=1, ncols=5, figsize=(15, 4), sharex=True)

        for i, col in enumerate(self.force_columns):
            ax = axes[i]
            ax.plot(time_normalized, self.force_data[col], color='green')
            ax.set_title(self.titles_muscles[i])
            if i == 0:
                ax.set_ylabel("Force (N)")
            ax.set_xlabel("% Gait Cycle")
            ax.grid(True)

        plt.tight_layout()
        
        if show:
            plt.show()

    def plot_multiple_trials(self, show=False):
        self.df_ik_list = []  # Store loaded DataFrames
        
        for subject in self.subjects:
            for trial in self.subjects[subject]:
                trial_obj = self.subjects[subject][trial]
                if trial_obj:
                    self.df_ik_list.append(trial_obj.ik.data)
                    
        for file in self.mot_files:  # Loop through each file
            with open(file, "r") as f:
                lines = f.readlines()

            # Load data using Pandas
            df = msk.pd.read_csv(file, delim_whitespace=True, skiprows=5)
            self.df_ik_list.append(df)

        # Normalize all loaded IK data
        self.df_ik_normalized_list = []  # Store normalized DataFrames

        for df in self.df_ik_list:  # Loop through each loaded DataFrame
            df_normalized = self.time_normalised_df(df=df)  # Apply normalization
            self.df_ik_normalized_list.append(df_normalized)  # Store normalized DataFrame

        # Ensure time is normalized to 101 points
        time_normalized = msk.np.linspace(0, 100, 101)

        # Select the specified columns from normalized data
        self.ik_data_list = []  # Store DataFrames with only the required columns

        for df_normalized in self.df_ik_normalized_list:  # Loop through each normalized DataFrame
            if set(self.ik_columns).issubset(df_normalized.columns):  # Check if columns exist
                self.ik_data_list.append(df_normalized[self.ik_columns])  # Select only specified columns
            else:
                print("Warning: Some specified columns are missing in a file.")

        # Plot mean and sd
        # Check if IK data exists
        if not self.ik_data_list:
            print("No IK data available to plot!")
        else:
            # Convert list of DataFrames to a single NumPy array
            combined_df = np.array([df.values for df in self.ik_data_list])  # Shape: (num_trials, num_timepoints, num_columns)

            # Check if data is properly structured
            if combined_df.shape[0] < 2:
                print("Not enough trials to calculate mean and standard deviation!")
            else:
                # Compute Mean and Standard Deviation
                mean_values = np.mean(combined_df, axis=0)
                std_values = np.std(combined_df, axis=0)

                # Normalize time from 0 to 100% Gait Cycle
                time_values = np.linspace(0, 100, combined_df.shape[1])

                # Create a shared figure for all subplots
                fig, axes = plt.subplots(nrows=1, ncols=len(self.ik_columns), figsize=(20, 5), sharex=True)

                if len(self.ik_columns) == 1:
                    axes = [axes]  # If only one column, ensure it's iterable

                for i, col in enumerate(self.ik_columns):
                    ax = axes[i]

                    # Plot mean line
                    ax.plot(time_values, mean_values[:, i], color='red', label="Mean", linewidth=2)

                    # Shade the standard deviation range
                    ax.fill_between(time_values, mean_values[:, i] - std_values[:, i],
                                    mean_values[:, i] + std_values[:, i], color='red', alpha=0.2, label="SD Range")

                    # Formatting
                    ax.set_title(col)
                    ax.set_xlabel("Gait Cycle (%)")
                    ax.set_xlim(0, 100)  # X-axis from 0% to 100% of the gait cycle
                    ax.grid(True)

                    # Set Y-label only for the first subplot
                    if i == 0:
                        ax.set_ylabel("Angle (Degrees)")
                        ax.legend()


                plt.tight_layout()

                if show:
                    plt.show()


## check if paths exist

In [40]:
os_analysis = openSim(leg='r', subjects=['PC002'], trials_to_load=['trial2'], trial_number=1)
os_analysis.subjects['PC002']['trial2_r1'].check_files()



File not found: c:\Git\opensim_tutorial\tutorials\repeated_sprinting\Simulations\PC002\trial2_r1\markers_experimental.trc
File not found: c:\Git\opensim_tutorial\tutorials\repeated_sprinting\Simulations\PC002\trial2_r1\processed_emg.mot
File not found: c:\Git\opensim_tutorial\tutorials\repeated_sprinting\Simulations\PC002\trial2_r1\inverse_dynamics.sto
File not found: c:\Git\opensim_tutorial\tutorials\repeated_sprinting\Simulations\PC002\trial2_r1\Results_SO_and_MA\PC002_StaticOptimization_force.sto
File not found: c:\Git\opensim_tutorial\tutorials\repeated_sprinting\Simulations\PC002\trial2_r1\Results_SO_and_MA\PC002_StaticOptimization_activation.sto
File not found: c:\Git\opensim_tutorial\tutorials\repeated_sprinting\Simulations\PC002\trial2_r1\joint_reacton_loads.sto
Error reading file: c:\Git\opensim_tutorial\tutorials\repeated_sprinting\Simulations\PC002\trial2_r1\GRF_Setup.xml
load() missing 1 required positional argument: 'xml_file'
File not found: c:\Git\opensim_tutorial\tutori

False

In [ ]:


os_analysis.subjects['PC002']['trial2_r1'].run_ID(osim_modelPath=os_analysis.subjects['PC002']['model'], 
                                                  coordinates_file=os_analysis.subjects['PC002']['trial2_r1'].ik.path, 
                                                  output_file=os_analysis.subjects['PC002']['trial2_r1'].id.path, 
                                                  external_loads_file=os_analysis.subjects['PC002']['trial2_r1'].grf_xml.path, 
                                                  LowpassCutoffFrequency=6, 
                                                  run_tool=True)

In [ ]:
os_analysis.subjects['PC002']['trial2_r1'].grf_xml.path

In [35]:
osim_modelPath=os_analysis.subjects['PC002']['model']
coordinates_file=os_analysis.subjects['PC002']['trial2_r1'].ik.path
output_file=os_analysis.subjects['PC002']['trial2_r1'].id.path
external_loads_file=os_analysis.subjects['PC002']['trial2_r1'].grf_xml.path
LowpassCutoffFrequency=6
run_tool=True

try: 
    model = osim.Model(osim_modelPath)
except Exception as e:
    print(f"Error loading model: {osim_modelPath}")
    print(e)

results_folder = os.path.dirname(output_file)

# Setup for excluding muscles from ID
exclude = osim.ArrayStr()
exclude.append("Muscles")
# Setup for setting time range
IKData = osim.Storage(coordinates_file)

# Create inverse dynamics tool, set parameters and run
id_tool = osim.InverseDynamicsTool()
id_tool.setModel(model)
id_tool.setCoordinatesFileName(coordinates_file)
id_tool.setExternalLoadsFileName(external_loads_file)
id_tool.setOutputGenForceFileName(output_file)
id_tool.setLowpassCutoffFrequency(LowpassCutoffFrequency)
id_tool.setStartTime(IKData.getFirstTime())
id_tool.setEndTime(IKData.getLastTime())
id_tool.setExcludedForces(exclude)
id_tool.setResultsDir(results_folder)
id_tool.printToXML(os.path.join(results_folder, "setup_ID.xml"))

if run_tool:
    id_tool.run()

RuntimeError: std::exception in 'bool OpenSim::InverseDynamicsTool::run()': Unknown exception

In [37]:
os.path.join(results_folder, "setup_ID.xml")

'c:\\Git\\opensim_tutorial\\tutorials\\repeated_sprinting\\Simulations\\PC002\\trial2_r1\\setup_ID.xml'

## Print trial to Json

In [ ]:
jsonFilePath = os.path.join(os_analysis.simulations_path, 'PC002', 'trial2_r1', 'settings.json')
trial = Trial(os.path.join(os_analysis.simulations_path, 'PC002', 'trial2_r1'))
trial.__dict__['jra'].__dict__

data = trial.__dict__
print(trial.name)

# with open(jsonFilePath, 'w') as f:
#     json.dump(data, f, indent=4)

# json_data = import_json_file(jsonFilePath)

# RunSO

In [ ]:

def run_SO(model_path, ik_path, actuators_file_path):
    '''
    Function to run Static Optimization using the OpenSim API.
    
    Inputs:
            modelpath(str): path to the OpenSim model file
            trialpath(str): path to the trial folder
            actuators_file_path(str): path to the actuators file
            
    '''
    
    trialpath = os.path.dirname(ik_path)   
    # create directories
    results_directory = os.path.relpath(trialpath, trialpath)
    coordinates_file =  os.path.relpath(trialpath, ik_path)
    modelpath_relative = os.path.relpath(model_path, trialpath)

    # create a local copy of the actuator file path and update name
    actuators_file_path = os.path.relpath(actuators_file_path, trialpath)

    # start model
    OsimModel = msk.osim.Model(modelpath_relative)

    # Get mot data to determine time range
    motData = msk.osim.Storage(coordinates_file)

    # Get initial and intial time
    initial_time = motData.getFirstTime()
    final_time = motData.getLastTime()

    # Static Optimization
    so = msk.osim.StaticOptimization()
    so.setName('StaticOptimization')
    so.setModel(OsimModel)

    # Set other parameters as needed
    so.setStartTime(initial_time)
    so.setEndTime(final_time)
    so.setMaxIterations(25)

    analyzeTool_SO = msk.classes.osimSetup.create_analysis_tool(coordinates_file,modelpath_relative,results_directory)
    analyzeTool_SO.getAnalysisSet().cloneAndAppend(so)
    analyzeTool_SO.getForceSetFiles().append(actuators_file_path)
    analyzeTool_SO.setReplaceForceSet(False)
    OsimModel.addAnalysis(so)

    analyzeTool_SO.printToXML(".\setup_so.xml")

    analyzeTool_SO = msk.osim.AnalyzeTool(".\setup_so.xml")

    trial = os.path.basename(trialpath)
    print(f"so for {trial}")

    # run
    analyzeTool_SO.run()



In [ ]:

run_SO(model_path=os_analysis.subjects['PC002']['model'],
        ik_path = os_analysis.subjects['PC002']['trial2'].ik.path,
        actuators_file_path = os_analysis.subjects['PC002']['trial2'].so_force.path)